In [ ]:
# |default_exp tasks
# |default_cls_lvl 3

In [ ]:
# |hide
%reload_ext autoreload
%autoreload 2

# tasks

Code for running experiments and hyper parameters optimization from the terminal

In [ ]:
# |export
from __future__ import annotations

import argparse, json, os, warnings
from functools import partial
from pathlib import Path

from dotenv import load_dotenv
from transformers import logging as hf_logging
import wandb

from course_copilot import utils, training, topic_segmentation, summarization

In [ ]:
# | hide
import pdb

from fastcore.test import *
import nbdev

from blurr.utils import print_versions

In [ ]:
# |export
# silence all the HF warnings and load environment variables
warnings.simplefilter("ignore")
hf_logging.set_verbosity_error()

os.environ["TOKENIZERS_PARALLELISM"] = "false"
load_dotenv()

try:
    from nbdev.imports import IN_NOTEBOOK
except:
    IN_NOTEBOOK = False

In [ ]:
# | echo: false
print("What we're running with at the time this documentation was generated:")
print_versions("torch fastai transformers")

What we're running with at the time this documentation was generated:
torch: 1.12.1+cu102
fastai: 2.7.9
transformers: 4.22.0


In [ ]:
# | export
def build_train_config(
    # Our task's `TrainConfig`
    train_config_cls: type,
    # The arguments (name and values) we want to update our `TrainConfig` with
    args,
) -> training.TrainConfig:
    train_config = train_config_cls()

    for arg in vars(args):
        if hasattr(train_config, arg):
            setattr(train_config, arg, getattr(args, arg))

    return train_config

In [ ]:
# | export
def run_experiment(
    # The ML task to run (e.g., topic_segmentation, summarization)
    task: str,
    # The name of your experiment (e.g., deberta_v3_large). This value is used in conjunction with `task` when
    # logging information with W&B or else saving data releveant to training/evaluation runs
    experiment_name: str,
    # Where the project's data is stored
    data_path: str = "data",
    # Where exported Learners and other models should stored
    model_output_path: str = "models",
    # Where any logged data should be stored
    log_output_path: str = "logs",
    # Whether predictions should be logged
    log_preds: bool = False,
    # The number of predictions that should be logged. It is left to each subclass to define what that means
    log_n_preds: int = None,
    # Whether or not to log experiments and sweeps to W&B
    use_wandb: bool = False,
    # Whether or not you want to have printed out everything during a training/evaulation run
    verbose: bool = False,
    # Any args/values we want to use to update our `TrainConfig` with
    args=None,
):
    if task == "topic_segmentation":
        train_config = build_train_config(topic_segmentation.TopicSegmentationConfig, args)

        trainer = topic_segmentation.TopicSegmentationModelTrainer(
            experiment_name=experiment_name,
            train_config=train_config,
            data_path=data_path,
            model_output_path=model_output_path,
            log_output_path=log_output_path,
            log_preds=log_preds,
            log_n_preds=log_n_preds,
            use_wandb=use_wandb,
            verbose=verbose,
        )
    elif task == "headline_summarization":
        train_config = build_train_config(summarization.SummarizationConfig, args)

        trainer = summarization.SummarizationModelTrainer(
            experiment_name=experiment_name,
            train_config=train_config,
            data_path=data_path,
            model_output_path=model_output_path,
            log_output_path=log_output_path,
            log_preds=log_preds,
            log_n_preds=log_n_preds,
            use_wandb=use_wandb,
            verbose=verbose,
        )

    elif task == "content_summarization":
        train_config = build_train_config(summarization.SummarizationConfig, args)

        trainer = summarization.SummarizationModelTrainer(
            experiment_name=experiment_name,
            train_config=train_config,
            data_path=data_path,
            model_output_path=model_output_path,
            log_output_path=log_output_path,
            log_preds=log_preds,
            log_n_preds=log_n_preds,
            use_wandb=use_wandb,
            verbose=verbose,
        )

    # run training
    results_df, raw_df, train_df, train_val_idxs = trainer.train()

In [ ]:
# | export
def prepare_tuning(
    # The ML task to run (e.g., topic_segmentation, summarization)
    task: str,
    # The name of your experiment (e.g., deberta_v3_large). This value is used in conjunction with `task` when
    # logging information with W&B or else saving data releveant to training/evaluation runs
    experiment_name: str,
    # Where the project's data is stored
    data_path: str,
    # Where exported Learners and other models should stored
    model_output_path: str,
    # Where any logged data should be stored
    log_output_path: str,
    # Whether predictions should be logged
    log_preds: bool,
    # The number of predictions that should be logged. It is left to each subclass to define what that means
    log_n_preds: int,
    # Whether or not to log experiments and sweeps to W&B
    use_wandb: bool,
    # Whether or not you want to have printed out everything during a training/evaulation run
    verbose: bool,
    # Any args/values we want to use to update our `TrainConfig` with
    args=None,
):
    if task == "topic_segmentation":
        train_config = build_train_config(topic_segmentation.TopicSegmentationConfig, args)

        trainer = topic_segmentation.TopicSegmentationModelTrainer(
            experiment_name=experiment_name,
            train_config=train_config,
            data_path=data_path,
            model_output_path=model_output_path,
            log_output_path=log_output_path,
            log_preds=log_preds,
            log_n_preds=log_n_preds,
            use_wandb=use_wandb,
            verbose=verbose,
        )
    elif task == "headline_summarization":
        pass
    elif task == "content_summarization":
        pass

    return trainer

In [ ]:
# | export
def add_required_args(parser):
    # define other `ModelTrainer` args
    parser.add_argument("--task", type=str, default="train", help="Options: train | tune")
    parser.add_argument("--experiment", type=str, default="test_experiment")
    parser.add_argument("--data_path", type=str, default="./data")
    parser.add_argument("--models_path", type=str, default="./models")
    parser.add_argument("--logs_path", type=str, default="./logs")
    parser.add_argument("--log_preds", type=bool, default=False)
    parser.add_argument("--log_n_preds", type=int, default=10)
    parser.add_argument("--use_wandb", type=bool, default=False)
    parser.add_argument("--sweep_config_fpath", type=str, default=None)
    parser.add_argument("--sweep_trials", type=int, default=20)
    parser.add_argument("--verbose", type=bool, default=False)

    return parser

In [ ]:
# | export
if __name__ == "__main__" and not IN_NOTEBOOK:
    train_config_attrs = [(k, type(v), v) for k, v in vars(training.TrainConfig).items() if not k.startswith("_")]

    headline_summarization_train_config_attrs = train_config_attrs
    content_summarization_train_config_attrs = train_config_attrs

    # instantiate argparser
    parser = argparse.ArgumentParser()
    subparsers = parser.add_subparsers(dest="subcommand", help="Sub-commands help")

    # instantiate argparser for topic segmentation
    parser_topic_segmentation = subparsers.add_parser("topic_segmentation", help="Topic segmentation")
    add_required_args(parser_topic_segmentation)
    train_config_attrs += [
        (k, type(v), v) for k, v in vars(topic_segmentation.TopicSegmentationConfig).items() if not k.startswith("_")
    ]
    for attr in train_config_attrs:
        parser_topic_segmentation.add_argument(f"--{attr[0]}", type=attr[1], default=attr[2])

    # # instantiate argparser for headline summarization
    parser_headline_summarization = subparsers.add_parser("headline_summarization", help="Headling summarization")
    add_required_args(parser_headline_summarization)
    headline_summarization_train_config_attrs += [
        (k, type(v), v) for k, v in vars(summarization.HeadlineSummarizationConfig).items() if not k.startswith("_")
    ]

    for attr in headline_summarization_train_config_attrs:
        parser_headline_summarization.add_argument(f"--{attr[0]}", type=attr[1], default=attr[2])

    #     # # instantiate argparser for content summarization
    #     parser_content_summarization = subparsers.add_parser("content_summarization", help="Content summarization")
    #     add_required_args(parser_content_summarization)
    #     content_summarization_train_config_attrs += [(k, type(v), v) for k, v in vars(summarization.ContentSummarizationConfig).items() if not k.startswith("_")]
    #     for attr in content_summarization_train_config_attrs:
    #         parser_content_summarization.add_argument(f"--{attr[0]}", type=attr[1], default=attr[2])

    # get the arg values
    args = parser.parse_args()
    task = args.subcommand

    # run the specific task task
    if task == "train":
        run_experiment(
            task,
            experiment_name=args.experiment,
            data_path=args.data_path,
            model_output_path=args.models_path,
            log_output_path=args.logs_path,
            log_preds=args.log_preds,
            log_n_preds=args.log_n_preds,
            use_wandb=args.use_wandb,
            verbose=args.verbose,
            args=args,
        )
    elif task == "tune":
        trainer = prepare_tuning(
            task,
            experiment_name=args.experiment,
            data_path=args.data_path,
            model_output_path=args.models_path,
            log_output_path=args.logs_path,
            log_preds=args.log_preds,
            log_n_preds=args.log_n_preds,
            use_wandb=args.use_wandb,
            verbose=args.verbose,
            args=args,
        )

        sweep_config_fpath = (
            f"./sweep_configs/{task}/default.json" if args.sweep_config_fpath is None else args.sweep_config_fpath
        )
        with open(sweep_config_fpath, "r") as f:
            sweep_config_d = json.load(f)

        # https://docs.wandb.ai/guides/sweeps/faq#how-can-i-change-the-directory-my-sweep-logs-to-locally
        os.environ["WANDB_DIR"] = args.logs_path

        # begin the sweep
        sweep_id = trainer.configure_sweep(sweep_config=sweep_config_d)
        wandb.agent(sweep_id, function=partial(trainer.train, sweep_config=sweep_config_d), count=args.sweep_trials)

## Export -

In [ ]:
# | hide
nbdev.nbdev_export()